In [7]:
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

import pandas as pd
import numpy as np
import time, datetime
import json

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Initilize Spotify

In [11]:
username = 'maguenza'
client_id = '8f5145e2a1574aa784b4e4b2dddeebc9'
client_secret = '2c9e490810e048aa91a4b168cc2521cb'


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Grab all albums for a specific artist

In [85]:
kanye = 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'

results = sp.artist_albums(kanye, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

album_id = []
for album in albums:
    #print(album['name'],',', album['id'])
    album_id.append(album['id'])

## For every album, grab songs

In [128]:
offset = 0
songs = []
for item in album_id:
    while True:
        content = sp.album_tracks(item,limit=50, offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break

## For all the songs, create a dictionary then transpose into a dataframe
- rename index into names then drop index column

In [149]:
new_dict = {item['name']:item for item in songs}

df_songs = pd.DataFrame(new_dict).T['id'].reset_index()
df_songs['name'] = df_songs['index']
del df_songs['index']

In [214]:
df_songs.head()

,id,name
0,4R5bSS8yoCl2czeWLr61aO,**Bonus Track** Diamonds From Sierra Leone
1,2CHmgtK8OCL28WtIK96u4N,30 Hours
2,5VzeI5JM2y9t21JwrWAnkH,Addiction
3,7rinBrI5ekbbOiLosWp65M,Addiction - Album Version (Edited)
4,2cYZpcIV39X48RnOFM7w2V,All Falls Down


In [219]:
song_id = []
for x in df_songs['id'].values:
    song_id.append(x)

## Now for all the song ids, find each song's feature and set to dataframe

In [221]:
index = 0
audio_features = []
while index < len(song_id):
    audio_features += sp.audio_features(song_id[index:index + 50])
    index += 50

features_list = []
for features in audio_features:
    features_list.append([features['energy'], features['liveness'],\
                          features['tempo'], features['speechiness'],\
                          features['acousticness'], features['instrumentalness'],\
                          features['time_signature'], features['danceability'],\
                          features['key'], features['duration_ms'],\
                          features['loudness'], features['valence'],\
                          features['mode'], features['type'],\
                          features['uri'],\
                          features['id']])

df = pd.DataFrame(features_list, columns=['energy', 'liveness',\
                                          'tempo', 'speechiness',\
                                          'acousticness', 'instrumentalness',\
                                          'time_signature', 'danceability',\
                                          'key', 'duration_ms', 'loudness',\
                                          'valence', 'mode', 'type', 'uri','id'])

In [222]:
df.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode,type,uri,id
0,0.864,0.269,96.483,0.291,0.154,0.000,4,0.419,7,238400,-3.372,0.512,1,audio_features,spotify:track:4R5bSS8yoCl2czeWLr61aO,4R5bSS8yoCl2czeWLr61aO
1,0.813,0.107,95.122,0.291,0.032,0.000,4,0.822,7,323303,-6.736,0.357,1,audio_features,spotify:track:2CHmgtK8OCL28WtIK96u4N,2CHmgtK8OCL28WtIK96u4N
2,0.761,0.227,115.085,0.187,0.178,0.000,4,0.550,1,267067,-8.224,0.536,0,audio_features,spotify:track:5VzeI5JM2y9t21JwrWAnkH,5VzeI5JM2y9t21JwrWAnkH
3,0.758,0.210,115.257,0.168,0.168,0.000,4,0.593,1,267067,-8.270,0.501,0,audio_features,spotify:track:7rinBrI5ekbbOiLosWp65M,7rinBrI5ekbbOiLosWp65M
4,0.717,0.152,90.893,0.506,0.184,0.000,4,0.643,8,223507,-4.876,0.460,0,audio_features,spotify:track:2cYZpcIV39X48RnOFM7w2V,2cYZpcIV39X48RnOFM7w2V


## Now mere song names and audio features on id to bring in song name

In [211]:
df_final = df.merge(df_songs, left_on = 'id', right_on = 'id', how = 'inner')

In [212]:
df_final.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode,type,uri,id,name
0,0.811,0.178,81.051,0.375,0.394,0.000,4,0.472,11,324440,-4.856,0.636,0,audio_features,spotify:track:0XMJORJKm4q3iorGAtcait,0XMJORJKm4q3iorGAtcait,Never Let Me Down
1,0.661,0.617,81.760,0.090,0.007,0.000,4,0.393,6,198960,-6.828,0.410,0,audio_features,spotify:track:3ZPEQbLCrOWIBg2JPNs3mZ,3ZPEQbLCrOWIBg2JPNs3mZ,Bring Me Down - Album Version (Edited)
2,0.396,0.187,79.132,0.942,0.346,0.000,4,0.641,1,31250,-11.897,0.940,1,audio_features,spotify:track:466XcSgGAiQxcDrcV0yoMP,466XcSgGAiQxcDrcV0yoMP,Skit #2 (Kanye West/Late Registration)
3,0.839,0.293,87.277,0.391,0.571,0.000,4,0.623,6,193733,-4.725,0.689,1,audio_features,spotify:track:1o1JjjwMHRCHcMa6aGIudK,1o1JjjwMHRCHcMa6aGIudK,Jesus Walks - Album Version (Edited)
4,0.579,0.517,86.522,0.103,0.198,0.002,4,0.383,1,547733,-3.765,0.103,0,audio_features,spotify:track:0jOL63hLMeSCHTXPdFkniq,0jOL63hLMeSCHTXPdFkniq,Runaway
